In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np

# 1. Veriyi pkl dosyasından yükle
file_path = 'hazirlanmis_etkilesim.pkl'
data = pd.read_pickle(file_path)

# İlk birkaç satırı görüntüle
print("Veri Setindeki İlk 5 Satır:")
print(data.head())

print("\n")



Veri Setindeki İlk 5 Satır:
    uye_id  urun_id           eklenme_tarihi  puan   event_type
0        0    72176  2024-08-05 05:43:02.000     1         view
1        0    67714  2024-08-05 05:45:51.000     5     purchase
2        0    66060  2024-08-05 05:53:56.000     4  add_to_cart
3  1659156    60388  2024-08-05 06:01:07.000     1         view
4        0    41669  2024-08-05 06:04:45.000     5     purchase




In [3]:
# 2. Categorical Encoding
urun_encoder = LabelEncoder()
data['urun_id_encoded'] = urun_encoder.fit_transform(data['urun_id'])

event_encoder = LabelEncoder()
data['event_type_encoded'] = event_encoder.fit_transform(data['event_type'])



In [4]:
# 3. Kullanıcı Bazlı Gruplama ve Sekans Oluşturma
grouped = data.groupby('uye_id').apply(
    lambda x: list(zip(x['urun_id_encoded'], x['event_type_encoded']))
).reset_index(name='sequences')



In [5]:
# 4. Sekansları Düzleştirme
# Sadece urun_id_encoded değerlerini kullanarak
sequences_flat = [[item[0] for item in seq] for seq in grouped['sequences']]



In [6]:
# 5. Sekansları Pad Etme (value=-1 yerine value=0 kullanıyoruz)
max_sequence_length = 10  # Maksimum sekans uzunluğu
padded_sequences = pad_sequences(
    sequences_flat,
    maxlen=max_sequence_length,
    padding='post',
    dtype='int32',
    value=0  # Padding için sıfır kullanılıyor
)



In [7]:
# 6. Giriş ve Çıkışları Hazırla
X = np.array([seq[:-1] for seq in padded_sequences])  # Girdi: Sekansın ilk kısımları
y = np.array([seq[-1] for seq in padded_sequences])  # Çıktı: Sekansın son öğesi



In [8]:
# 7. Modeli Tanımla
model = Sequential([
    Embedding(input_dim=len(urun_encoder.classes_), output_dim=64, input_length=X.shape[1]),  # Ürün sayısı kadar unique ID
    LSTM(64, return_sequences=False),  # LSTM katmanı
    Dense(32, activation='relu'),  # Tam bağlı katman
    Dense(len(urun_encoder.classes_), activation='softmax')  # Çıkış katmanı (ürün sınıfları kadar)
])



In [10]:
# 8. Modeli Derle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 9. Modeli Eğit
model.fit(X, y, epochs=10, batch_size=32)


Epoch 1/10


633/633 [==============================] - 7s 6ms/step - loss: 0.7934 - accuracy: 0.9378
Epoch 2/10
633/633 [==============================] - 3s 6ms/step - loss: 0.2402 - accuracy: 0.9458
Epoch 3/10
633/633 [==============================] - 4s 6ms/step - loss: 0.1957 - accuracy: 0.9618
Epoch 4/10
633/633 [==============================] - 4s 6ms/step - loss: 0.1712 - accuracy: 0.9679
Epoch 5/10
633/633 [==============================] - 4s 7ms/step - loss: 0.1598 - accuracy: 0.9692
Epoch 6/10
633/633 [==============================] - 4s 6ms/step - loss: 0.1514 - accuracy: 0.9697
Epoch 7/10
633/633 [==============================] - 4s 6ms/step - loss: 0.1441 - accuracy: 0.9713
Epoch 8/10
633/633 [==============================] - 4s 6ms/step - loss: 0.1358 - accuracy: 0.9726
Epoch 9/10
633/633 [==============================] - 4s 7ms/step - loss: 0.1285 - accuracy: 0.9741
Epoch 10/10
633/633 [==============================] - 4s 6ms/step - loss: 0.1215 - accuracy: 0.9

In [11]:
# Kullanıcıdan verilen ürün ID'sini al
input_urun_id = 67714  # Örnek: Verilen ürün ID'si

# Verilen ürün ID'sini encode et
input_encoded = urun_encoder.transform([input_urun_id])[0]

# Model giriş formatına uygun hale getirmek için bir sekans oluştur
# (Bu durumda, ürün ID'si ile tek bir sekans oluşturuyoruz)
input_sequence = np.array([[input_encoded]])

# Tahmin yap
predicted_encoded = np.argmax(model.predict(input_sequence), axis=1)

# Tahmin edilen encoded ürün ID'sini orijinal ürün ID'sine çevir
predicted_urun_id = urun_encoder.inverse_transform(predicted_encoded)

print(f"Verilen Ürün ID: {input_urun_id}, Tahmin Edilen Öneri Ürün ID: {predicted_urun_id[0]}")


1/1 [==============================] - 0s 467ms/step
Verilen Ürün ID: 67714, Tahmin Edilen Öneri Ürün ID: 73914


In [16]:
def catalog_coverage_dynamic(recommendations, all_items):

    # Önerilen tüm ürünlerin kümesini oluştur
    recommended_items = set(item for user_recommendations in recommendations for item in user_recommendations)
    
    # Catalog coverage oranını hesapla
    coverage = len(recommended_items) / len(all_items) if len(all_items) > 0 else 0.0
    return coverage

# Tüm benzersiz ürünleri belirle
all_items = set(data['urun_id_encoded'])

# Modelden öneriler alalım (her kullanıcı için öneri yapıyoruz)
recommendations = []
for user_sequence in X:
    user_input = user_sequence.reshape(1, -1)  # Kullanıcı girişini hazırlama
    predicted_class = np.argmax(model.predict(user_input), axis=1)  # Model tahmini
    recommendations.append(predicted_class.tolist())  # Önerilen ürünleri ekle

# Catalog Coverage hesapla
coverage = catalog_coverage_dynamic(recommendations, all_items)
print(f"Catalog Coverage: {coverage:.4f}")


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 7ms/step


...

1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 10ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 10ms/step
Catalog Coverage: 0.0865


In [18]:
def mean_number_of_suggestions(recommendations):
    # Her kullanıcının önerilen ürün sayısını hesapla
    num_suggestions_per_user = [len(user_recommendations) for user_recommendations in recommendations]
    
    # Ortalama öneri sayısını hesapla
    mns = np.mean(num_suggestions_per_user) if len(num_suggestions_per_user) > 0 else 0.0
    return mns

# Örnek Kullanım:
# Kullanıcı bazında önerilen ürünlerin listesi
# Öneriler modelden alınır
recommendations = []
for user_sequence in X:
    user_input = user_sequence.reshape(1, -1)
    predicted_class = np.argmax(model.predict(user_input, verbose=0), axis=1)
    recommendations.append(predicted_class.tolist())

# MNS hesapla
mns = mean_number_of_suggestions(recommendations)
print(f"Mean Number of Suggestions (MNS): {mns:.2f}")



Mean Number of Suggestions (MNS): 1.00


In [9]:
def mean_diversity_of_suggestions(recommendations):

    # Önerilen ürünlerin toplam setini hesapla
    all_recommended_items = set(item for user_recommendations in recommendations for item in user_recommendations)
    
    # Her kullanıcı için önerilen benzersiz ürünlerin oranını hesapla
    diversity_per_user = [
        len(set(user_recommendations)) / len(user_recommendations) if len(user_recommendations) > 0 else 0
        for user_recommendations in recommendations
    ]
    
    # Çeşitlilik oranlarının ortalamasını al
    mds = np.mean(diversity_per_user) if len(diversity_per_user) > 0 else 0.0
    return mds

# Örnek Kullanım:
# Öneriler modelden alınır
recommendations = []
for user_sequence in X:
    user_input = user_sequence.reshape(1, -1)
    predicted_class = np.argmax(model.predict(user_input, verbose=0), axis=1)
    recommendations.append(predicted_class.tolist())

# MDS hesapla
mds = mean_diversity_of_suggestions(recommendations)
print(f"Mean Diversity of Suggestions (MDS): {mds:.2f}")


Mean Diversity of Suggestions (MDS): 1.00


In [10]:
def mean_percentage_of_suggestions(recommendations, total_items):
    # Her kullanıcı için öneri yüzdesini hesapla
    percentage_per_user = [
        (len(set(user_recommendations)) / len(total_items)) * 100 if len(total_items) > 0 else 0
        for user_recommendations in recommendations
    ]
    
    # Ortalama yüzdeyi hesapla
    mps = np.mean(percentage_per_user) if len(percentage_per_user) > 0 else 0.0
    return mps

# Örnek Kullanım:
# Öneriler modelden alınır
recommendations = []
for user_sequence in X:
    user_input = user_sequence.reshape(1, -1)
    predicted_class = np.argmax(model.predict(user_input, verbose=0), axis=1)
    recommendations.append(predicted_class.tolist())

# Tüm önerilebilir ürünleri belirle
total_items = set(data['urun_id_encoded'])

# MPS hesapla
mps = mean_percentage_of_suggestions(recommendations, total_items)
print(f"Mean Percentage of Suggestions (MPS): {mps:.4f}")



Mean Percentage of Suggestions (MPS): 0.0297


In [13]:
def average_precision(relevant_items, recommended_items):

    score = 0.0
    hits = 0
    for i, item in enumerate(recommended_items):
        if item in relevant_items:
            hits += 1
            score += hits / (i + 1)  # Precision@K hesaplama
    return score / len(relevant_items) if relevant_items else 0.0


def mean_average_precision(relevant_items_list, recommendations_list):

    average_precisions = [
        average_precision(relevant_items, recommended_items)
        for relevant_items, recommended_items in zip(relevant_items_list, recommendations_list)
    ]
    return np.mean(average_precisions) if average_precisions else 0.0

# Örnek Kullanım:
# Öneriler modelden alınır
recommendations_list = []
for user_sequence in X:
    user_input = user_sequence.reshape(1, -1)
    predicted_class = np.argmax(model.predict(user_input, verbose=0), axis=1)
    recommendations_list.append(predicted_class.tolist())

# Kullanıcıların ilgili ürünleri (ground truth)
relevant_items_list = data.groupby('uye_id')['urun_id_encoded'].apply(set).tolist()

# MAP hesapla
map_score = mean_average_precision(relevant_items_list, recommendations_list)
print(f"Mean Average Precision (MAP): {map_score:.4f}")


Mean Average Precision (MAP): 0.0001


In [19]:
import pickle

# Tüm metrikleri tek bir sözlükte topla
rnn_results = {
    "MAP": map_score,
    "CC": coverage,
    "MNS": mns,
    "MDS": mds,
    "MPS": mps
}

# Tek bir dosyaya kaydet
with open("rnn_sonuclari.pkl", "wb") as f:
    pickle.dump(rnn_results, f)
